# Home Sales Analysis

Note: this notebook was not used but is filled out for refrence of how to do outside of Google Colab. 

### Step 1: Import Libraries

In [ ]:
# Optional: to install virtualenv
!pip install virtualenv

# Import findspark and initialize.
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

### Step 2: Read the Data

In [ ]:
# Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
url_df = pd.read_csv(url)

In [ ]:
# Create a temporary view of the DataFrame.
df = spark.createDataFrame(url_df)
df.createOrReplaceTempView("home_sales")
df.show(5)

### Step 3: Analyze and Answer Using SparkSQL 

In [ ]:
# Query 1: What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query_1 = """
SELECT year(date) AS year_sold, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year_sold
ORDER BY year_sold
"""

# Execute the query and show the results.
result_1 = spark.sql(query_1)
result_1.show()

In [ ]:
# Query 2: What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query_2 = """
SELECT date_built AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query and show the results
result_2 = spark.sql(query_2)
result_2.show()

In [ ]:
# Query 3: What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query_3 = """
SELECT date_built AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query and show the results.
result_3 = spark.sql(query_3)
result_3.show()

In [ ]:
# Query 4: What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query_4 = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""

# Execute the query and show the results.
result_4 = spark.sql(query_4)
result_4.show()

# Print the run time.
end_time = time.time()
print("Query 4 run time: %s seconds" % (end_time - start_time))

In [ ]:
# Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

In [ ]:
# Check if the table is cached.
print("Is the table cached?", spark.catalog.isCached("home_sales"))

In [ ]:
# Run Query 4 again using the cached table and measure runtime 
start_time = time.time()
result_4_cached = spark.sql(query_4)
result_4_cached.show()

# Print the run time.
end_time = time.time()
print("Query 4 cached run time: %s seconds" % (end_time - start_time))

In [ ]:
# Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("home_sales_partitioned.parquet")

# Read the formatted parquet data.
parquet_df = spark.read.parquet("home_sales_partitioned.parquet")
parquet_df.show()

In [ ]:
# Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [ ]:
# Run Query 4 on the Parquet data and measure runtime
start_time = time.time()
result_4_parquet = spark.sql(query_4)
result_4_parquet.show()

# Print the run time.
end_time = time.time()
print("Query 4 parquet run time: %s seconds" % (end_time - start_time))

In [ ]:
# Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

In [ ]:
# Check if the home_sales is no longer cached
print("Is the table cached?", spark.catalog.isCached("home_sales"))

Resources: Data for this dataset was generated by _edX Boot Camps LLC_, and is intended for educational purposes only.